In [181]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time

from scipy.ndimage.filters import convolve
from tqdm import tqdm

In [182]:
# install tqdm
#pip install tqdm

In [183]:
# install cv2
#pip install opencv-python

In [184]:
#install scipy
#pip install scipy

In [185]:
# prevents overflow for uint8
def overflow_8(img):
    img[img>=255] = 255
    img[img<=0] = 0

    return img

In [186]:
def gauss(n=3, sigma=1):
    r = np.arange(0, n, dtype=np.float32) - (n-1.)/2.
    r = np.exp(-r**2./(2.*sigma**2))

    return r / np.sum(r)

In [187]:
def gauss2d(shape=(3,3), sigma=1):
    g1 = gauss(shape[0], sigma).reshape([shape[0], 1])
    g2 = gauss(shape[1], sigma).reshape([1, shape[1]])

    return np.matmul(g1,g2)

In [189]:
def psf2otf(flt, img_shape):
    flt_top_half = flt.shape[0]//2
    flt_bottom_half = flt.shape[0] - flt_top_half
    flt_left_half = flt.shape[1]//2
    flt_right_half = flt.shape[1] - flt_left_half
    
    flt_padded = np.zeros(img_shape, dtype=flt.dtype)
    flt_padded[:flt_bottom_half, :flt_right_half] = flt[flt_top_half:, flt_left_half:]
    flt_padded[:flt_bottom_half, img_shape[1]-flt_left_half:] = flt[flt_top_half:, :flt_left_half]
    flt_padded[img_shape[0]-flt_top_half:, :flt_right_half] = flt[:flt_top_half, flt_left_half:]
    flt[:flt_top_half, :flt_left_half]

    return flt_padded

In [192]:
# sharpen filter in spatial domain filtering
def unsharp_masking_0(img, alpha=1, sigma=1):
    kernel = gauss2d(shape=(3,3), sigma=sigma)
    
    start_time = time.time() # record start time
    
    img_convolved = convolve(img, kernel)
    result = img + alpha*(img - img_convolved)
    
    end_time = time.time() # record end time
    time_elapsed = end_time-start_time
    
    return result, time_elapsed

In [193]:
# sharpen filter in frequency domain filtering 
def unsharp_masking_1(img, alpha=1, sigma=1):
    kernel = gauss2d(shape=(3,3), sigma=sigma)
    kernel = psf2otf(kernel, img.shape)

    start_time = time.time() # record start time
    
    img_fft = np.fft.fft2(img)
    kernel_fft = np.fft.fft2(kernel)
    
    img_multiplied = img_fft * kernel_fft
    
    result_freq = img_fft + alpha*(img_fft - img_multiplied)
    result = np.abs(np.fft.ifft2(result_freq))
    
    end_time = time.time() # record end time
    time_elapsed = end_time-start_time
    
    return result, time_elapsed

In [194]:
def img_pad(img, padding = 0):
    height, width = img.shape
    img_padded = np.zeros((img.shape[0]+2*padding, img.shape[1]+2*padding))
    
    img_padded[padding:(padding+height), padding:(padding+width)] = img

    return img_padded

In [195]:
def bilateral_filter(img, diameter=3, sigma_s=1000, sigma_r=50):
    height, width = img.shape # (434, 640)
    output = np.zeros((height, width))
    
    img_padded = img_pad(img, padding=diameter//2)
    height_p, width_p = img_padded.shape
    
    for n in tqdm(range(height_p)):
        if n-(diameter//2)<0 or n+(diameter//2)>=height_p:
            continue
        for m in range(width_p):
            if m-(diameter//2)<0 or m+(diameter//2)>=width_p:
                continue
            filtered_img = 0
            norm_factor = 0
            for l in range(-(diameter//2), (diameter//2)+1):
                for k in range(-(diameter//2), (diameter//2)+1):
                    # Gaussian Kernel
                    g = np.exp(-(k**2 + l**2)/(2*(sigma_s**2)))
                    # Edge-aware Kernel
                    r = np.exp((-(np.abs(img_padded[n+l][m+k]-img_padded[n][m]))**2)/(2*(sigma_r**2)))
                    kernel = g * r
                    
                    filtered_img += (img_padded[n+l][m+k] * kernel)
                    norm_factor += kernel
            
            filtered_img = filtered_img / norm_factor
            output[n-(diameter//2)][m-(diameter//2)] = filtered_img
    
    return output

In [196]:
#img = cv2.imread('mj.jpg', cv2.IMREAD_GRAYSCALE).astype(np.float32)
img = cv2.imread('obama.jpg', cv2.IMREAD_GRAYSCALE).astype(np.float32)

In [197]:
unsharp_0, time_elapsed = unsharp_masking_0(img, alpha=1, sigma=1)
result_unsharp_0 = overflow_8(unsharp_0).astype('uint8')

print(time_elapsed)

cv2.imshow('result_unsharp_0', result_unsharp_0)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite("result_unsharp_000.jpg", result_unsharp_0)

0.009973526000976562


True

In [198]:
unsharp_1, time_elapsed = unsharp_masking_1(img, alpha=1, sigma=1)
result_unsharp_1 = overflow_8(unsharp_1).astype('uint8')

print(time_elapsed)

cv2.imshow('result_unsharp_1', result_unsharp_1)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite("result_unsharp_1.jpg", result_unsharp_1)

0.12077116966247559


True

In [199]:
bilateral = bilateral_filter(img, 5, sigma_s=20, sigma_r=20)

result_bilateral = overflow_8(bilateral).astype('uint8')

cv2.imshow('result_bilateral', result_bilateral)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite("result_bilateral_20_20.jpg", result_bilateral)

100%|██████████| 618/618 [00:48<00:00, 12.73it/s]


True